# Que es JuMP?

JuMP es un _lenguaje de modelamiento_ para problemas de optimización, escrito en Julia. 

Supongamos que queremos resolver un modelo sencillo de optimización. Por ejemplo:

$$
\begin{align*}
\max_{x_1,x_2,x_3} \quad& x_1 + 2x_2 + 3x_3 \\
\text{s.t.}\quad& -x_1 + x_2 + x_3 \leq 20 \\
& x_1 -3x_2 + x_3 \leq 30 \\
& 0 \leq x_1 \leq 40 \\
& x_2, x_3 \geq 0.
\end{align*}
$$


PAra resolver esto, se requiere usar un _solver_: una implementación de software de distintos métodos de optimización (Simplex, B&B, etc). Tipicamente, tienen interfaces de programación que no son tan sencillos. Por ejemplo, el modelo anterior en CPLEX para Java sería:

```java
import ilog.concert.*;
import ilog.cplex.*;
public class Example {
 public static void main(String[] args) {
   try {
    IloCplex cplex = new IloCplex();
double[] lb = {0.0, 0.0, 0.0};
double[] ub = {40.0, Double.MAX_VALUE, Double.MAX_VALUE}; IloNumVar[] x = cplex.numVarArray(3, lb, ub);
    double[] objvals = {1.0, 2.0, 3.0};

cplex.addMaximize(cplex.scalProd(x, objvals));
    cplex.addLe(cplex.sum(cplex.prod(-1.0, x[0]),
                   cplex.prod( 1.0, x[1]),
                   cplex.prod( 1.0, x[2])), 20.0);
    cplex.addLe(cplex.sum(cplex.prod( 1.0, x[0]),
                   cplex.prod(-3.0, x[1]),
                   cplex.prod( 1.0, x[2])), 30.0);
if ( cplex.solve() ) {
cplex.out().println("Solution status = " + cplex.getStatus()); cplex.out().println("Solution value = " + cplex.getObjValue());
     double[] val = cplex.getValues(x);
     int ncols = cplex.getNcols();
     for (int j = 0; j < ncols; ++j)
       cplex.out().println("Column: " + j + " Value = " + val[j]);
    }
    cplex.end();
   }
   catch (IloException e) {
    System.err.println("Concert exception '" + e + "' caught");
} }
}
```

Un lenguaje de modelamiento (como JuMP) permite escribir el modelo de opotimización de una forma mas natural, encargandose Julia de transformarlo en el formato _bajo-nivel_ requerido por cada solver.

Existen otros lenguajes de modelamiento (como GAMS, AMPL, etc).  Por que JuMP? 

* Fácil para el usuario
* Mejor performance
* Independiente de un solver específico
* Por ser un lenjuage de programación, facil de extender para hacer cosas mas complicadas.

Para más información de JuMP, ver: [JuMP docs](https://jump.readthedocs.io/en/latest/index.html).

En esta sesión, veremos JuMP para problemas simples de optimización.


# Un primer ejemplo
Transformemos este modelo de dos variables a JuMP.
$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

Primero, cargamos JuMP y un solver (en este caso, Clp)

In [ ]:
using JuMP, Clp

Ahora contruimos un _Model_, que contendrá el modelo. con todos sus detalles (restricciones objetivos, variables).

In [ ]:
model = Model(solver=ClpSolver())

Ahora definimos dos variables, con la macreo ``@variable``. El primer argumento es el model al cual agregar la variable, el segundo parametro define el nombre y cotas de esta. 

In [ ]:
@variable(model, x >= 0)
@variable(model, y >= 0)

In [ ]:
model

Agregamos una restricción con la macro ``@constraint``. LA escribimos de forma algebraica, tal como en el modelo original. 

In [ ]:
@constraint(model, x + y <= 1)
model

Y el objetico con la macro ``@objective``.

In [ ]:
@objective(model, Max, x + 2y)

In [ ]:
model

Para resolverlo (con el _solver_ CBC), usamos la función ``solve`` aplicada al modelo que queremos resolver.

In [ ]:
solve(model)

La función retorna el estado del modelo resuelto (en este caso, ``:Optimal``). Ahora podemos ver el valor óptimo de las variables y el objetivo:

In [ ]:
@show getvalue(x)
@show getvalue(y)
@show getobjectivevalue(model)

# Ejercicio

Construya y resuelva el siguiente modelo

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

# Revenue Management para redes de aviones.

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

En este problema, debemos decidir cuantos tickets vender para cada origen-destino (O-D) a cada precio (en este caso, dos precios, reducido y normal).  El objetivo es maximizar el retorno, y no podemos vender mas tickets que la demanda, o que el número de asientos que tiene el aviob.

## Problema con 3 viajes

Resolvamos un problema de juguete, con 3 pares origen-destino, y dos clases de precio (normal y reducido). Las tres rutas posibles son BOS-MDW, MDW-SFO, o BOS-SFO via MDW. BOS es Boston, MDW es Chicago-Midway, y SFO es San Francisco. La data para cada ruta es la siguiente:

```
Capacidad del Avión: 166

BOS-MDW
        Regular  Descontado
Ganancia:  428      190
Demanda: 80       120

BOS-SFO
        Regular  Descontado
Ganancia:  642      224
Demanda: 75       100

MDW-SFO
        Regular  Descontado
Ganancia:  512      190
Demanda: 60       110
```

In [ ]:
nrm = Model(solver=ClpSolver())

In [ ]:
@variables(nrm, begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end)
nrm

In [ ]:
@objective(nrm, Max, 428BOStoMDW_R + 190BOStoMDW_D +
                     642BOStoSFO_R + 224BOStoSFO_D +
                     512MDWtoSFO_R + 190MDWtoSFO_D)
nrm

In [ ]:
@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
nrm

In [ ]:
status = solve(nrm)
status  

In [ ]:
@show getvalue(BOStoMDW_R)
@show getvalue(BOStoMDW_D)
@show getobjectivevalue(nrm)

Podemos agregar mas aeropuertos, pero hagasmolo de forma inteligente.

## Conjuntos de variables y sumas en JuMP

Construyamos ahora un _conjunto_ de variables. Es decir, una variable ``x`` pero que está indexada, por ejempplo, de  1 a 10:

In [ ]:
m = Model()
@variable(m, x[1:10] >= 0)

El conjunto de indices es lo que va dentro de los parentesis ``[...]``. Se pueden crear mutiples indices, simplemente separando por coma cada conjunto:


In [ ]:
@variable(m, y[1:10,["red","blue"]] <= 1)

Para expresiones mas complicadas, podemos nombrar los indices y usarlos en el resto de la definición:


$$
i \leq z_{ij} \leq ub_j \;\;\; \forall i \in \{1,...,10\}, j \in \{i+1, ..., 10\}
$$

In [ ]:
ub = rand(10)
@variable(m, i <= z[i=1:10,j=(i+1):10] <= ub[j])

También podemos poner condiciones lógicas dentro del bloque ``[...]``, separadas por un punto y coma ``;``:

In [ ]:
@variable(m, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Ahora que ya sabemos usar conjuntos de variables, nos gustaría usarlas en restricciones (posiblemente, también definidas como un conjunto). Para esto, necesitamos expresar sumas de multiples indices, con condiciones. Para eso, usaremos  ``sum(...)``. El primer argumento es lo que se suma, seguido de un ``for`` para expresar los subindices sobre los cuales se suma. Esto puede estar seguido de multiples indices (separados con ``,``) y  condiciones logicas que se escriben despues de ``if``.

$$ \sum _{i = 1}^{10} x_i \leq 1$$

In [ ]:
@constraint(m, sum(x[i] for i in 1:10) <= 1)

$$ 
\begin{equation}
\sum_{\substack{i\in\{1,...,10\}\\
                c\in\{"red","blue"\}}}
       coef(c) \cdot y_{ic} = 1
\end{equation}
$$

In [ ]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(m, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

$$ 
\begin{equation}
\sum_{i = 1}^{10} \sum_{j = i+1}^{10} 
       i \cdot j \cdot z_{ij} \leq
\sum_{\substack{i\in\{1,...,10\},
                c\in\{"red","blue"\} \\
                \text{s.t. } iseven(i) \text{ or } c = "red"}}
       i^2 \cdot w_{ic} 
\end{equation}
$$

In [ ]:
@constraint(m, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

## Extensión a más aeropuertos

Volvamos al ejemplo de los aeropuertos, y hagamos una formulación nueva que use esta forma general de definir restricciones. Supondremos una tarifa única. Modificaremos el problema un poco, pensando en que la demanda es en realidad el valor esperado de clientes que llegarán.  Para esto, asumimos uns probabilidad fija $p_v$ de que un cliente compra un pasaje de un vuelo $v$ en un periodo, y un número de períodos $T$. De esa forma:
$$ E\left(Demanda_v\right) = p_v\cdot T $$

Veamos los datos primero:

<img src="red.png" alt="Red de Vuelos" style="width: 400px;"/>


In [ ]:
# Veamos los viajes y destinos
origenes = [:BOS, :JFK, :ATL, :MDW]
destinos = [:SFO, :LAX, :SEA, :MDW]
# y los vuelos
segmentos = [(:BOS, :MDW), (:JFK, :MDW), (:ATL, :MDW), (:MDW,:SFO), (:MDW,:LAX), (:MDW,:SEA)]

# cada vuelo (entre cada origen y destino) tiene una tarifa asociada,
# y una probabilidad de que en un "periodo" llegue un cliente pidiendo ese vuelo.
# anotaremos eso con diccionarios:
probab = Dict()
revenue = Dict()

probab[[(:BOS,:MDW)]] = 0.06  ; revenue[[(:BOS,:MDW)]] = 40;
probab[[(:JFK,:MDW)]] = 0.096 ; revenue[[(:JFK,:MDW)]] = 30;
probab[[(:ATL,:MDW)]] = 0.046 ; revenue[[(:ATL,:MDW)]] = 30;
probab[[(:MDW,:SFO)]] = 0.073 ; revenue[[(:MDW,:SFO)]] = 10;
probab[[(:MDW,:LAX)]] = 0.159 ; revenue[[(:MDW,:LAX)]] = 40;
probab[[(:MDW,:SEA)]] = 0.067 ; revenue[[(:MDW,:SEA)]] = 10;
probab[[(:BOS,:MDW),(:MDW,:SFO)]] = 0.043 ; revenue[[(:BOS,:MDW),(:MDW,:SFO)]] = 190
probab[[(:BOS,:MDW),(:MDW,:LAX)]] = 0.019 ; revenue[[(:BOS,:MDW),(:MDW,:LAX)]] = 80
probab[[(:BOS,:MDW),(:MDW,:SEA)]] = 0.112 ; revenue[[(:BOS,:MDW),(:MDW,:SEA)]] = 90
probab[[(:JFK,:MDW),(:MDW,:SFO)]] = 0.075 ; revenue[[(:JFK,:MDW),(:MDW,:SFO)]] = 70
probab[[(:JFK,:MDW),(:MDW,:LAX)]] = 0.031 ; revenue[[(:JFK,:MDW),(:MDW,:LAX)]] = 60
probab[[(:JFK,:MDW),(:MDW,:SEA)]] = 0.044 ; revenue[[(:JFK,:MDW),(:MDW,:SEA)]] = 190
probab[[(:ATL,:MDW),(:MDW,:SFO)]] = 0.012 ; revenue[[(:ATL,:MDW),(:MDW,:SFO)]] = 65
probab[[(:ATL,:MDW),(:MDW,:LAX)]] = 0.021 ; revenue[[(:ATL,:MDW),(:MDW,:LAX)]] = 50
probab[[(:ATL,:MDW),(:MDW,:SEA)]] = 0.113 ; revenue[[(:ATL,:MDW),(:MDW,:SEA)]] = 10

# Vuelos
#vuelos = keys(probab)
vuelos = [[(:JFK, :MDW), (:MDW, :SFO)],
  [(:ATL, :MDW), (:MDW, :SFO)],
  [(:JFK, :MDW)],
  [(:BOS, :MDW)],
  [(:MDW, :SEA)],
  [(:ATL, :MDW), (:MDW, :LAX)],
  [(:BOS, :MDW), (:MDW, :LAX)],
  [(:MDW, :LAX)],
  [(:ATL, :MDW), (:MDW, :SEA)],
  [(:BOS, :MDW), (:MDW, :SEA)],
  [(:ATL, :MDW)],
  [(:JFK, :MDW), (:MDW, :LAX)],
  [(:MDW, :SFO)],
  [(:JFK, :MDW), (:MDW, :SEA)],
  [(:BOS, :MDW), (:MDW, :SFO)]]


# finalmente, fijemos la capacidad (20 para cada vuelo) y el numero de periodos T
T = 100
capac= Dict()
for leg in segmentos
    capac[leg]=20
end


Ahora hagamos nuestro modelo. Tenemos que tener una variable para vuelo (de origen a fin)

In [ ]:
nrm2 = Model(solver=ClpSolver())

@variable(nrm2, 0 <= x[v=vuelos] <= probab[v]*T)
nrm2

El objetivo es maximizar el revenue total, sobre todos los vuelos

In [ ]:
@objective(nrm2, Max, sum(revenue[v]*x[v] for v in vuelos))
nrm2

### Ejercicio:
agregue la restricción de que no podemos sobrepasar la capacidad de los aviones en cada segmento.

In [ ]:
@defConstrRef legCapConstr[segmentos]
for leg in segmentos
        legCapConstr[leg] = ### COMPLETAR ACA LA RESTRICCIÓN
    end
nrm2

y resolvemos el problema

In [ ]:
# Now solve the model
solve(nrm2)
@show getvalue(x)
@show getobjectivevalue(nrm2)

Lo importante de este problema no son tanto los asientos reservados para cada tramo, sino los *duales* de las restricciones de capacidad.  Estos me reflejan cual es el costo marginal de cada tramo.

Para recuperar los duales, usamos la funcion ``getDual``

In [ ]:
d = getdual(legCapConstr)

## Ejercicio:
Transformemos lo anterior en una función, que recibe la capacidad de cada tramo ``capac`` y el número de periodos ``T`` y calcula este óptimo, retornando los duales para cada tramo 

In [ ]:
function NRM(capac,T)
    ## Complete la función.
    return getdual(legCapConstr)
end

In [ ]:
NRM(capac,100)

# Online Network Revenue Management
En la realidad, este problema es "online". En cada período, un pasajero llega, y busca un vuelo. ¿Le ofrecemos el pasaje o no?
- Si le aceptamos el pasaje, tendremos el revenue de ese pasaje, pero perdimos capacidad para un pasajero.
- Si lo rechazamos, guardamos la capacidad, pero perdimos el revenue de ese cliente.

Lo mas sencillo que podemos hacer es una regla glotona. Cada vez que llega un pasajero, lo aceptamos si hay capacidad en los vuelos que quiere. ¿Que sucedería? Supongamos que simulamos 100 dias... 

In [ ]:
capacidad_original = Dict()
for leg in segmentos
    capacidad_original[leg] = 20
end

capacidad_actual = capacidad_original;
revenue_total = 0
for t in 1:100
    for v in vuelos
        if (rand() < probab[v])
            # El pasajero quiere tomar el vuelo! Veamos si hay capacidad
            hay_capacidad = true
            for leg in v
                if capacidad_actual[leg] == 0
                    hay_capacidad = false
                end
            end
            if hay_capacidad
                # Hay capacidad obtenemos su revenue y descontamos 1 asiento de cada segmento
                revenue_total += revenue[v]
                for leg in v
                    capacidad_actual[leg] -= 1
                end
            end
        end
    end
end
println("Revenue total = $revenue_total")
capacidad_actual


## Ejercicio:  
Repita lo anterior 1000 veces, y plotee la densidad (`Geom.density` en ``Gadfly``) del revenue que obtendría.

## Bid-price 
Esta regla dice que aceptamos un pasaje solamente si el revenue que entrega es mayor que el costo marginal de los tramos que ocupa. 

### Ejercicio:
Modifique la simulación anterior, para usar la función que creamos antes que entrega los duales, y usar el criterio del _Bid-price_ para decidir si vender un asiento o no.  ¿Es mejor este criterio que el glotón?